In [1]:
from sigvisa.graph.sigvisa_graph import SigvisaGraph
from sigvisa.signals.common import Waveform





from sigvisa.source.event import get_event

from sigvisa.infer.run_mcmc import run_open_world_MH

from sigvisa.infer.mcmc_logger import MCMCLogger

In [2]:
"""
sg = SigvisaGraph(template_model_type="dummyPrior", template_shape="lin_polyexp",
                      wiggle_model_type="dummy", wiggle_family="dummy",
                      phases="leb", nm_type = "ar")
evid = 5335822
wave = Waveform(data=np.zeros(20), stime=1240241314.33, srate=5.0, sta="MKAR", chan="BHZ", filter_str="freq_0.8_4.5")
wn = sg.add_wave(wave)
ev = get_event(evid=evid)
evnodes = sg.add_event(ev, observed=True)"""
"""logger = MCMCLogger(write_template_vals=True, dump_interval=100)
run_open_world_MH(sg, steps=5000,
                  enable_template_moves=True,
                  enable_event_moves=True,
                  logger=logger,
                  enable_event_openworld=False,
                  enable_template_openworld=False)"""
"""def build_relocation_sg_leb(evids, stas):
    sg = SigvisaGraph(template_model_type="dummyPrior", template_shape="lin_polyexp",
                      wiggle_model_type="dummy", wiggle_family="dummy",
                      phases="leb", nm_type = "ar")
    
    for evid in evids:
        for sta in stas:
            wave = load_event_station_chan(evid, sta, "auto", exclude_other_evs=True)
            sg.add_wave(wave)
    for evid in evids:
        ev = get_event(evid)
        sg.add_event(ev, observe=True)
    return sg

    sg = SigvisaGraph(template_model_type="dummyPrior", template_shape="lin_polyexp",
                      wiggle_model_type="dummy", wiggle_family="dummy",
                      phases="leb", nm_type = "ar")"""

'def build_relocation_sg_leb(evids, stas):\n    sg = SigvisaGraph(template_model_type="dummyPrior", template_shape="lin_polyexp",\n                      wiggle_model_type="dummy", wiggle_family="dummy",\n                      phases="leb", nm_type = "ar")\n    \n    for evid in evids:\n        for sta in stas:\n            wave = load_event_station_chan(evid, sta, "auto", exclude_other_evs=True)\n            sg.add_wave(wave)\n    for evid in evids:\n        ev = get_event(evid)\n        sg.add_event(ev, observe=True)\n    return sg\n\n    sg = SigvisaGraph(template_model_type="dummyPrior", template_shape="lin_polyexp",\n                      wiggle_model_type="dummy", wiggle_family="dummy",\n                      phases="leb", nm_type = "ar")'

In [3]:

from sigvisa.synthetic.doublets import *

def sample_events(basedir, seed=0):
    n_evs = 10
    lons = [129, 130]
    lats = [-3.5, -4.5]
    times = [1238889600, 1245456000]
    mbs = [4.0, 5.0]
    sw = SampledWorld(seed=seed)
    sw.sample_region_with_doublet(n_evs, lons, lats, times, mbs)
    sw.stas = ["MK31", "AS12", "CM16", "FITZ", "WR1"]
    gpcov = GPCov([0.7,], [ 40.0, 5.0],
                  dfn_str="lld",
                  wfn_str="compact2")
    param_means = build_param_means(sw.stas)
    sw.set_basis(wavelet_family="db4_2.0_3_30", iid_repeatable_var=0.1,
                  iid_nonrepeatable_var=0.4, srate=5.0)
    sw.joint_sample_arrival_params(gpcov, param_means)
    sw.sample_signals("freq_0.8_4.5")

    wave_dir = os.path.join(basedir, "sampled_%d_spreadtime" % seed)
    sw.serialize(wave_dir)
    #sw.train_gp_models_true_data()
    #sw.save_gps(wave_dir, run_name="synth_truedata")
    return sw


import os
basedir = os.path.join(os.getenv("SIGVISA_HOME"), "experiments", "synth_wavematch")
#sw = sample_events(basedir)

wave_dir = os.path.join(basedir, "sampled_%d_spreadtime" % 0)
sw = load_sampled_world(wave_dir)
dummyPrior = dict([(param, Gaussian(sw.param_means["MK31"][param], std=np.sqrt(sw.gpcov.wfn_params[0]))) for param in sw.param_means["MK31"].keys()
])

In [4]:
import copy

def corrupt_ev(ev, stddevs):
    ev = copy.copy(ev)
    ev.lon = ev.lon + np.random.randn() * stddevs['lon']
    ev.lat = ev.lat + np.random.randn() * stddevs['lat']
    ev.depth = ev.depth + np.random.randn() * stddevs['depth']
    ev.time = ev.time + np.random.randn() * stddevs['time']
    ev.mb = ev.mb + np.random.randn() * stddevs['mb']
    return ev

def set_true_templates(sg, sw):
    for sta in sw.tm_params.keys():
        for i in range(len(sw.evs)):
            tmnodes = sg.get_template_nodes(eid=i+1, sta=sta, phase=sw.phase, chan=sw.chans[sta], band=sw.band)
            for param in sw.tm_params[sta]:
                k, n = tmnodes[param]
                n.set_value(sw.tm_params[sta][param][i])

sg = SigvisaGraph(template_model_type="dummyPrior", template_shape="lin_polyexp",
                  wiggle_model_type="gp_joint", wiggle_family="db4_2.0_3_30",
                  phases=["P",], nm_type = "ar", runids=(-1,), joint_wiggle_prior=(0.01, sw.gpcov))

for i in sw.waves.keys():
    for sta in sw.waves[i].keys():
        wn = sg.add_wave(sw.waves[i][sta])
        basis, iid_std, target_coef = wn.wavelet_basis
        wn.wavelet_basis = (basis, sw.scaled, target_coef)
        
stddevs = {"lon": 0.2, "lat": 0.2, "depth": 20.0, "time": 3.0, "mb": 0.3}

for ev in sw.evs:
    sg.add_event(ev, observed=ev, stddevs=stddevs)
    #sg.add_event(ev, observed=corrupt_ev(ev, stddevs), stddevs=stddevs)
    
set_true_templates(sg, sw)

In [ ]:
logger = MCMCLogger(write_template_vals=False, dump_interval=5)

corrupted_evs = []
for i,ev in enumerate(sw.evs):
    eid = i+1
    lon = sg.all_nodes["%d;lon_obs" %eid ].get_value()
    lat = sg.all_nodes["%d;lat_obs" %eid ].get_value()
    depth = sg.all_nodes["%d;depth_obs" %eid ].get_value()
    mb = sg.all_nodes["%d;mb_obs" %eid ].get_value()
    time = sg.all_nodes["%d;time_obs" %eid ].get_value()
    cev = Event(lon=lon, lat=lat, depth=depth, mb=mb, time=time, eid=eid)
    corrupted_evs.append(cev)
    
with open(os.path.join(logger.run_dir, "obs_events.pkl"), "wb") as f:
    pickle.dump(corrupted_evs, f)

with open(os.path.join(logger.run_dir, "events.pkl"), "wb") as f:
    pickle.dump(sw.evs, f)
    
run_open_world_MH(sg, steps=1000,
                  enable_template_moves=True,
                  enable_event_moves=True,
                  logger=logger,
                  enable_event_openworld=False,
                  enable_template_openworld=False)